In [1]:
# 變數前處理

%run preprocessing.py
print ("shape of usersXprod :", usersXprod.shape)

p = pd.read_csv("products.csv")
p = p.drop(["product_name"], axis=1)
usersXprod = pd.merge(usersXprod, p, how='left', on=['product_id'])


import data
merge orders and order_products__prior
size of prd features : 5
size of users features : 13
nb of usersXproducts features : 13
merge prod, user and usersXprod feature on usersXprod
order row 2000000
order row 4000000
order row 6000000
order row 8000000
order row 10000000
order row 12000000
shape of usersXprod : (13307953, 32)


In [2]:
# train and test

X_test = usersXprod.loc[usersXprod.eval_set == "test", :].copy()
X_test.drop(['eval_set'], axis=1, inplace = True)

train = usersXprod.loc[usersXprod.eval_set == "train", :].copy()
train.drop(['eval_set'], axis=1, inplace = True)


In [3]:
# train set convert to form of submit

train_details = ordert[["user_id", "order_id", "product_id", "reordered"]]

try:
    df_train_gt = pd.read_csv('train.csv', index_col='order_id')
except:
    train_gtl = []

    for uid, subset in train_details.groupby('user_id'):
        subset1 = subset[subset.reordered == 1]
        oid = subset.order_id.values[0]

        if len(subset1) == 0:
            train_gtl.append((oid, 'None'))
            continue

        ostr = ' '.join([str(int(e)) for e in subset1.product_id.values])
        # .strip is needed because join can have a padding space at the end
        train_gtl.append((oid, ostr.strip()))

    df_train_gt = pd.DataFrame(train_gtl)

    df_train_gt.columns = ['order_id', 'products']
    df_train_gt.set_index('order_id', inplace=True)
    df_train_gt.sort_index(inplace=True)
    
    df_train_gt.to_csv('train.csv')

In [4]:
from catboost import CatBoostClassifier as cbc
import time

def catboost_cv(X_train, y_train, X_val, y_val, features_to_use):
    cb = cbc(iterations = 100,
             thread_count = 8, 
             verbose = True,
             random_seed = 42,
             learning_rate = 0.05,
             use_best_model = True,
             depth = 8,
             fold_permutation_block_size = 64,
             calc_feature_importance = True,
             leaf_estimation_method = 'Gradient')

    cb.fit(X = X_train[features_to_use].values,
           y = y_train.values, 
           cat_features = [X_train[features_to_use].columns.get_loc('aisle_id')], 
           eval_set = [X_val[features_to_use].values, y_val.values],
           verbose = True,
           #plot=True
          )
    return cb

# 計時
class tick_tock:
    def __init__(self, process_name, verbose=1):
        self.process_name = process_name
        self.verbose = verbose
    def __enter__(self):
        if self.verbose:
            print(self.process_name + " begin ......")
            self.begin_time = time.time()
    def __exit__(self, type, value, traceback):
        if self.verbose:
            end_time = time.time()
            print(self.process_name + " end ......")
            print('time lapsing {0} s \n'.format(end_time - self.begin_time))

# 整理成submit形式
def ka_add_groupby_features_n_vs_1(df, group_columns_list, target_columns_list, methods_list, keep_only_stats=True, verbose=1):
    '''Create statistical columns, group by [N columns] and compute stats on [1 column]

       Parameters
       ----------
       df: pandas dataframe
          Features matrix
       group_columns_list: list_like
          List of columns you want to group with, could be multiple columns
       target_columns_list: list_like
          column you want to compute stats, need to be a list with only one element
       methods_list: list_like
          methods that you want to use, all methods that supported by groupby in Pandas

       Return
       ------
       new pandas dataframe with original columns and new added columns

       Example
       -------
       ka_add_stats_features_n_vs_1(train, group_columns_list=['x0'], target_columns_list=['x10'])
    '''
    with tick_tock("add stats features", verbose):
        dicts = {"group_columns_list": group_columns_list , "target_columns_list": target_columns_list, "methods_list" :methods_list}

        for k, v in dicts.items():
            try:
                if type(v) == list:
                    pass
                else:
                    raise TypeError(k + "should be a list")
            except TypeError as e:
                print(e)
                raise

        grouped_name = ''.join(group_columns_list)
        target_name = ''.join(target_columns_list)
        combine_name = [[grouped_name] + [method_name] + [target_name] for method_name in methods_list]

        df_new = df.copy()
        grouped = df_new.groupby(group_columns_list)

        the_stats = grouped[target_name].agg(methods_list).reset_index()
        the_stats.columns = [grouped_name] + \
                            ['_%s_%s_by_%s' % (grouped_name, method_name, target_name) \
                             for (grouped_name, method_name, target_name) in combine_name]
        if keep_only_stats:
            return the_stats
        else:
            df_new = pd.merge(left=df_new, right=the_stats, on=group_columns_list, how='left')
        return df_new

# prediction compare with train
def compare_results(df_gt, df_preds):
    
    df_gt_cut = df_gt.loc[df_preds.index]
    
    f1 = []
    for gt, pred in zip(df_gt_cut.sort_index().products, df_preds.sort_index().products):
        lgt = gt.replace("None", "-1").split(' ')
        lpred = pred.replace("None", "-1").split(' ')

        rr = (np.intersect1d(lgt, lpred))
        precision = np.float(len(rr)) / len(lpred)
        recall = np.float(len(rr)) / len(lgt)

        denom = precision + recall
        f1.append(((2 * precision * recall) / denom) if denom > 0 else 0)
    
    return(np.mean(f1))



In [5]:
# cross-validation
# thershold = 0.202
# convert to form of submit

df_cvfolds = []
cb = []
for fold in range(4):
    train_subset = train[train.user_id % 4 != fold]
    valid_subset = train[train.user_id % 4 == fold]

    X_train = train_subset.drop('reordered', axis=1)
    y_train = train_subset.reordered

    X_val = valid_subset.drop('reordered', axis=1)
    y_val = valid_subset.reordered

    val_index = X_val[['user_id', 'product_id', 'order_id']]
    
    features_to_use = list(X_train.columns)
    features_to_use.remove('user_id')
    features_to_use.remove('product_id')
    features_to_use.remove('order_id')

    cb.append(catboost_cv(X_train, y_train, X_val, y_val, features_to_use))
    rawpreds = cb[-1].predict_proba(X_val[features_to_use].values)

    lim = .202
    val_out = val_index.copy()

    val_out.loc[:, 'reordered'] = (rawpreds[:,1] > lim).astype(int)
    val_out.loc[:, 'product_id'] = val_out.product_id.astype(str)
    presubmit = ka_add_groupby_features_n_vs_1(val_out[val_out.reordered == 1], 
                                               group_columns_list = ['order_id'],
                                               target_columns_list = ['product_id'],
                                               methods_list = [lambda x: ' '.join(set(x))],
                                               keep_only_stats = True)

    presubmit = presubmit.set_index('order_id')
    presubmit.columns = ['products']

    fullfold = pd.DataFrame(index = val_out.order_id.unique())

    fullfold.index.name = 'order_id'
    fullfold['products'] = ['None'] * len(fullfold)

    fullfold.loc[presubmit.index, 'products'] = presubmit.products

    print(fold, compare_results(df_train_gt, fullfold))

    df_cvfolds.append(fullfold)

Borders generated
0:	learn 0.6350273768	test 0.6349723465	bestTest 0.6349723465		total: 53.4s	remaining: 1h 28m 9s
1:	learn 0.5846795989	test 0.5845703567	bestTest 0.5845703567		total: 1m 17s	remaining: 1h 3m 28s
2:	learn 0.5419232682	test 0.5417262977	bestTest 0.5417262977		total: 1m 43s	remaining: 55m 42s
3:	learn 0.5046411156	test 0.5043733238	bestTest 0.5043733238		total: 2m 7s	remaining: 51m 4s
4:	learn 0.4726605466	test 0.4723389751	bestTest 0.4723389751		total: 2m 33s	remaining: 48m 29s
5:	learn 0.4446332698	test 0.4442886982	bestTest 0.4442886982		total: 3m	remaining: 47m 7s
6:	learn 0.4203329539	test 0.4199755293	bestTest 0.4199755293		total: 3m 25s	remaining: 45m 24s
7:	learn 0.399087304	test 0.3987097604	bestTest 0.3987097604		total: 6m 42s	remaining: 1h 17m 12s
8:	learn 0.3807681134	test 0.3803732151	bestTest 0.3803732151		total: 7m 10s	remaining: 1h 12m 34s
9:	learn 0.3649281875	test 0.3645250806	bestTest 0.3645250806		total: 7m 34s	remaining: 1h 8m 14s
10:	learn 0.3514975

In [6]:
df_cv = pd.concat(df_cvfolds)
print(compare_results(df_train_gt, df_cv))

0.378174243812


In [8]:
cb_full = cbc(iterations = 100, 
              thread_count = 8, 
              verbose = True, 
              random_seed = 42,
              learning_rate = 0.05,
              depth = 8,
              fold_permutation_block_size = 64,
              calc_feature_importance = True,
              leaf_estimation_method = 'Gradient')

cb_full.fit(X = train[features_to_use].values, 
            y = train.reordered.values,
            cat_features = [train[features_to_use].columns.get_loc('aisle_id')],
            verbose = True,
           #plot=True
           )

Borders generated
0:	learn 0.6349675088	total: 1m 16s	remaining: 2h 6m 1s
1:	learn 0.584790297	total: 1m 58s	remaining: 1h 36m 41s
2:	learn 0.5416821913	total: 2m 43s	remaining: 1h 28m 15s
3:	learn 0.5046547768	total: 3m 27s	remaining: 1h 22m 59s
4:	learn 0.4722999747	total: 4m 9s	remaining: 1h 19m 2s
5:	learn 0.4444803667	total: 4m 48s	remaining: 1h 15m 17s
6:	learn 0.420466089	total: 5m 29s	remaining: 1h 12m 55s
7:	learn 0.3993038514	total: 6m 11s	remaining: 1h 11m 13s
8:	learn 0.3809734197	total: 6m 51s	remaining: 1h 9m 16s
9:	learn 0.3652435498	total: 7m 31s	remaining: 1h 7m 46s
10:	learn 0.3517755551	total: 8m 13s	remaining: 1h 6m 31s
11:	learn 0.3402141839	total: 8m 53s	remaining: 1h 5m 9s
12:	learn 0.3300555532	total: 9m 32s	remaining: 1h 3m 53s
13:	learn 0.3209402158	total: 10m 12s	remaining: 1h 2m 44s
14:	learn 0.3129941312	total: 10m 53s	remaining: 1h 1m 43s
15:	learn 0.3059925761	total: 11m 35s	remaining: 1h 52s
16:	learn 0.2999866163	total: 12m 18s	remaining: 1h 5s
17:	lear

In [9]:
testpreds = X_test[['user_id', 'product_id', 'order_id']].copy()
testpreds['reordered'] = (cb_full.predict_proba(X_test[features_to_use].values)[:,1] > .202).astype(int)
testpreds.product_id = testpreds.product_id.astype(str)

g = testpreds[testpreds.reordered == 1].groupby('order_id', sort=False)
df_testpreds = g[['product_id']].agg(lambda x: ' '.join(set(x)))

# complete (but empty) test df
df_test = pd.DataFrame(index = X_test.order_id.unique())
df_test.index.name = 'order_id'
df_test['products'] = ['None'] * len(df_test)
print (len(df_test), len(df_testpreds))

# combine empty output df with predictions
df_test.loc[df_testpreds.index, 'products'] = df_testpreds.product_id
df_test.sort_index(inplace=True)
df_test.to_csv('submit_catboost.csv')


75000 72053
